# Chapter 6. Odds and Addends
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap06.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import DataFrames as Dfs
import Distributions as Dsts
import CairoMakie as Cmk

In [ ]:
include("./pmf.jl")
import .ProbabilityMassFunction as pmf

### Odds

Odds [odds = $\frac{P(success)}{P(failure)} = \frac{P(success)}{1 -
P(success)}$], are used in the form of so called **Bayes's Rule**.

Bayes’s Rule is convenient if you want to do a Bayesian update on paper or in
your head. It also sheds light on the important idea of evidence and how we can
quantify the strength of evidence.